In [ ]:
import torch_ECT as poly
import numpy as np
from shape_reader import ShapeReader
import torch
# Read in shapes
s1=ShapeReader.shape_from_file('data/polyhedron.off',prepare=True)
s2=ShapeReader.shape_from_file('data/polyhedron.off',prepare=False)

In [ ]:
#Rotate one of the shapes
import polygon_ECT as polyx
from scipy.spatial.transform import Rotation as R
tmp=R.random(random_state=100)
rotmatrix=tmp.as_matrix()
s2.V=np.matmul(s2.V,rotmatrix)
s2.prepare()

In [ ]:
# Define some helper functions and metrics
def torch_return_ECT(s1):
    tmp=[]
    for key in s1.clean_polygon_gains:
        TMP=torch.tensor(s1.clean_polygon_gains[key])
        for j in range(TMP.shape[0]):
            megatmp=[]
            megatmp.append(TMP[j])
            megatmp.append(torch.tensor(s1.V[key,:], dtype=torch.float64))
            polyg = s1.polygon_angles[key][s1.clean_polygons[key][j]]
            if len(polyg) < 3: continue
            polyg = poly.poly_orientation(polyg)
            megatmp.append(torch.tensor(polyg, dtype=torch.float64))
            megatmp.append(key)
            N = poly.return_N(polyg)
            megatmp.append(torch.tensor(N, dtype=torch.float64))
            tmp.append(megatmp)
    return tmp  
ECT1=torch_return_ECT(s1)
ECT2=torch_return_ECT(s2)
def rotate_ECT(ECT,matrix):
    ECT_tmp=list()
    for i in range(len(ECT)):
        tmp=list()
        tmp.append(ECT[i][0])
        tmp.append(torch.matmul(ECT[i][1],matrix))
        tmp.append(torch.matmul(ECT[i][2],matrix))
        tmp.append(ECT[i][3])
        tmp.append(torch.matmul(ECT[i][4],matrix))
        ECT_tmp.append(tmp)
    return(ECT_tmp)
def rotation_matrix2(v):
    matrix=torch.zeros([3,3],dtype=torch.double)
    matrix[0,:]=torch.stack((torch.cos(v[1])*torch.cos(v[2]),torch.sin(v[0])*torch.sin(v[1])*torch.cos(v[2])-torch.cos(v[0])*torch.sin(v[2]),torch.cos(v[0])*torch.sin(v[1])*torch.cos(v[2])+torch.sin(v[0])*torch.sin(v[2])))
    matrix[1,:]=torch.stack((torch.cos(v[1])*torch.sin(v[2]),torch.sin(v[0])*torch.sin(v[1])*torch.sin(v[2])+torch.cos(v[0])*torch.cos(v[2]),torch.cos(v[0])*torch.sin(v[1])*torch.sin(v[2])-torch.sin(v[0])*torch.cos(v[2])))
    matrix[2,:]=torch.stack((-torch.sin(v[1]),torch.sin(v[0])*torch.cos(v[1]),torch.cos(v[0])*torch.cos(v[1])))
    return(matrix)
def forward(x,angles):
    matrix=rotation_matrix2(angles)
    ECT1=rotate_ECT(x,matrix)
    return(ECT1)
def criterion(target, ECT):
    return(poly.ECT_distance_d(target,ECT))
target=ECT2
data=ECT1

In [ ]:
import math
def SO3_distance(angles):
    matrix=R.from_euler('xyz',angles).as_matrix()
    diff=np.matmul(rotmatrix,matrix.transpose())
    upstairs=np.sum(np.diag(diff))-1
    return(np.arccos(upstairs/2))

In [ ]:
# Perform gradient ascent
print(tmp.as_euler('xyz')) # The truth
vector=torch.tensor([0.0,0.0,0.0],requires_grad=True)
lamda=1
iter=10
for i in range(iter):
    Y_pred=forward(data,vector)
    loss=criterion(Y_pred,target)
    loss.backward()
    vector.data=vector.data + lamda*vector.grad.data/torch.pow(torch.sum(torch.pow(vector.grad.data,2)),0.5)
    print('Estimate:')
    print(vector.detach().numpy())
    print('Distance from truth:')
    print(SO3_distance(vector.detach().numpy()))
    vector.grad.data.zero_()